# ONE REST queries

While the basic `one.search` command allows us to filter experimental sessions by date, lab,
subject etc., you may quickly find that you want to apply more complex queries to restrict your
search output. This can be achieved using the `one.alyx.rest` command. This has access to a range
of tables stored in the database and gives us more flexibility when forming our queries.

<div class="alert alert-info">
Note.

REST queries can only be made while ONE is in online mode.
</div>

## `one.search` vs `one.alyx.rest`
We will get started by importing ONE

In [1]:
from one.api import ONE
one = ONE(base_url='https://openalyx.internationalbrainlab.org')

We can list the possible tables that we have access to, by typing the following command

In [2]:
one.alyx.list_endpoints()

['brain-regions',
 'cache',
 'cache.zip',
 'channels',
 'chronic-insertions',
 'data-formats',
 'data-repository',
 'data-repository-type',
 'dataset-types',
 'datasets',
 'downloads',
 'fields-of-view',
 'files',
 'fov-location',
 'imaging-stack',
 'insertions',
 'labs',
 'locations',
 'new-download',
 'notes',
 'procedures',
 'projects',
 'register-file',
 'revisions',
 'sessions',
 'subjects',
 'surgeries',
 'sync-file-status',
 'tags',
 'tasks',
 'trajectories',
 'uploaded',
 'users',
 'water-administrations',
 'water-requirement',
 'water-restricted-subjects',
 'water-restriction',
 'water-type',
 'weighings']

Let's first look at `sessions` table. The information about what filters can be applied to this
table are available under FILTERS
[here](https://openalyx.internationalbrainlab.org/docs/#sessions-list). Let's apply a filter based
on `performance_qte` and `task_protocol`. We can do this using the following expression,

In [3]:
sess_info = one.alyx.rest('sessions', 'list', performance_gte=70, task_protocol='ephys')
sess_info[0]

{'id': 'ae8787b1-4229-4d56-b0c2-566b61a25b77',
 'subject': 'NR_0027',
 'start_time': '2022-08-23T09:26:09',
 'number': 1,
 'lab': 'steinmetzlab',
 'projects': ['ibl_neuropixel_brainwide_01'],
 'url': 'https://openalyx.internationalbrainlab.org/sessions/ae8787b1-4229-4d56-b0c2-566b61a25b77',
 'task_protocol': '_iblrig_tasks_ephysChoiceWorld6.6.2'}

Notice how this command doesn't just return the session eID but a dictionary containing information
about each session. We can extract the set of eIDs using the `to_eid` method:

In [4]:
one.to_eid(sess_info[0])

'ae8787b1-4229-4d56-b0c2-566b61a25b77'

You might have noticed that this same query could have been achieved using the `one.search` method
with the [remote query flag](../one_modes/one_modes.html),

In [5]:
eids = one.search(performance_gte=70, task_protocol='ephys', query_type='remote')
eids

## Accessing other Alyx tables

With the `one.alyx.rest` command we are not only limited to the `sessions` table, but can formulate
queries based on other tables to find session eIDs of interest. Consider the case where we want to
find all sessions that have probe insertions that target a specific ML and AP coordinate. For this
we can formulate our query based on the
[trajectories table](https://openalyx.internationalbrainlab.org/docs/#trajectories-list).
Let's see if there are any probe insertions at the coordinates ML(x) = -2225, and AP(y) = -1894
from bregma.

In [6]:
trajs = one.alyx.rest('trajectories', 'list', x=953.7, y=-1533.4)
trajs[0]

{'id': '05588582-01c9-4201-880a-8fb73ea8acea',
 'probe_insertion': '6d3b68e0-3efd-4b03-b747-16e44118a0a9',
 'x': 953.7,
 'y': -1533.4,
 'z': -211.1,
 'depth': 6683.4,
 'theta': 17.0,
 'phi': 0.0,
 'roll': 0.0,
 'provenance': 'Micro-manipulator',
 'session': {'subject': 'CSH_ZAD_001',
  'start_time': '2020-01-16T15:53:21.500926',
  'number': 1,
  'lab': 'zadorlab',
  'id': '3e7ae7c0-fe8b-487c-9354-036236fa1010',
  'task_protocol': '_iblrig_tasks_ephysChoiceWorld6.2.5'},
 'probe_name': 'probe00',
 'coordinate_system': None,
 'datetime': '2020-06-09T07:59:14.315700',
 'json': None,
 'chronic_insertion': None}

We can find the session eID associated with this trajectory by looking at the `id` of the `session`
field in the returned dictionary

In [7]:
eid = trajs[0]['session']['id']
eid

'3e7ae7c0-fe8b-487c-9354-036236fa1010'

<div class="alert alert-info">

Note.

It is not just sessions that have unique IDs associated with them! Every object stored in Alyx has
a unique UUID, whether it is a trajectory, a subject, a user or a dataset. For example in the above
example we can access the unique ID of the trajectory by typing `traj_id = trajs[0]['id']`


</div>

## Searching with `one.alyx.rest`
The `one.alyx.rest` command is also provides an alternative method to `one.list` for searching
the database for possible keywords that you can use to restrict your queries. For example, if we
 want to find the names of female subjects in the Witten lab that are alive, we can use the
 [subjects table](https://openalyx.internationalbrainlab.org/docs/#subjects-list) to write,

In [8]:
subj_info = one.alyx.rest('subjects', 'list', lab='cortexlab', sex='F', alive=True)
subj_nickname = [subj['nickname'] for subj in subj_info]
subj_nickname

['KS004', 'KS005', 'KS017', 'KS018', 'KS019', 'KS023', 'KS024', 'KS025']